# Figure panel 4

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # Data 
            library(tidyverse), 
            library(data.table), 


            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(ComplexHeatmap), 
            library(circlize), 
            library(viridis), 
            library(ggplotify), 
            library(ggrepel), 
            library(cowplot), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
# py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source
source("bin/so_pl.R")
source("bin/tradeseq_pp.R")
source("bin/tradeseq_pl.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

# Load reference data 

In [ ]:
tf <- read.table("/research/peer/fdeckert/reference/animaltfdb/Mus_musculus_TF.txt")[[2]]

lr <- CellChat::CellChatDB.mouse[[1]] %>% dplyr::select(pathway_name, ligand, receptor.symbol, receptor.family, annotation) %>% dplyr::filter(annotation %in% c("Cell-Cell Contact", "Secreted Signaling", "ECM-Receptor")) %>% 
    separate_rows(receptor.symbol, sep=", ") %>% dplyr::distinct() %>%
    separate_rows(ligand, sep=", ") %>% dplyr::distinct() %>% 
    dplyr::rename(pathway=pathway_name, receptor=receptor.symbol, family=receptor.family)

# Load data

## scRNAseq data

In [ ]:
so <- readRDS("data/scRNAseq/object/pp_1.rds")
so <- NormalizeData(so)

## Erythroid lineage fitgam and make contrasts

In [ ]:
tradeseq_res <- readRDS("result/lineage/tradeseq_res_2.rds")
fitgam <- tradeseq_res[["fitgam"]]
ptpg <- tradeseq_res[["ptpg"]]

In [ ]:
contrast_1 <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1")
ptpg_res_1 <- ptpg[ptpg[["contrast"]]==paste0(contrast_1, collapse=":"), ]

contrast_2 <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D3")
ptpg_res_2 <- ptpg[ptpg[["contrast"]]==paste0(contrast_2, collapse=":"), ]

contrast_3 <- c("IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
ptpg_res_3 <- ptpg[ptpg[["contrast"]]==paste0(contrast_3, collapse=":"), ]

# UMAP erytime

In [ ]:
dpt <- read.csv("result/pseudotime/ery_dpt.csv", row.names=1)
so <- AddMetaData(so, dpt)

In [ ]:
fp_1 <- fplot(so, features="dpt_pseudotime", pt_size=0.25, option="F", size_select=4)
fp_1[[1]]$data <- fp_1[[1]]$data[!is.na(fp_1[[1]]$data$feature), ]

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/umap_erytime.pdf", width=3.5, height=2.5)

gridExtra::grid.arrange(
    
    fp_1 %>% egg::set_panel_size(., width=unit(4.5, "cm"), height=unit(4.5, "cm"))

)

dev.off()

# Erythroid progenitor marker 

All BFU-E potential in the cKit+ CD71low/Ter119low population resided in the CD150+ (Slamf1) fraction. Exclusion of mature erythroid precursor cells in spleen was facilitated by inclusion of CD24a as a negative selection marker. Of the surface markers tested (CD9, CD11a, CD34, CD48, CD63, and CD79b), only CD9 could further fractionate the BFU-E-containing CD150+ population in stressed spleen. To further discriminate putative multi-potent stress-progenitors from lineage restricted stress-BFU-E we also included Sca1, which in steady-state BM separates Sca1+ (Ly6a) hematopoietic stem cells (HSC) and Sca1– myelo-erythroid progenitors.

## Module score 

In [ ]:
so <- readRDS("data/scRNAseq/object/pp.rds")
so <- NormalizeData(so)

In [ ]:
fate_simplex_fwd <- read.csv("result/cellrank/fate_simplex_fwd_p.csv", row.names=1) 
fate_simplex_fwd <- fate_simplex_fwd[rownames(fate_simplex_fwd) %in% colnames(so), ]
colnames(fate_simplex_fwd) <- c("CR_1", "CR_2")

In [ ]:
so[["CR"]] <- SeuratObject::CreateDimReducObject(embeddings=as.matrix(fate_simplex_fwd), key="CR_", assay="RNA")

In [ ]:
so <- so[, so$celltype_low %in% c("GMP", "NeuP", "BasoP", "MastP", "MegP", "MEP", "Proerythroblast")]

In [ ]:
features <- list(

    msP_1=c("Hlf+", "Esam+"), 
    msP_2=c("Kit+", "Flt3+"), 
    msP_3=c("Cd24a+", "Tfrc+"), 
    msP_4=c("Slamf1+"),
    msP_5=c("Fcgr2b+", "Fcgr3+"), 
    msP_6=c("Ms4a2+"),  
    msP_7=c("Elane+", "Prtn3+"), 
    msP_8=c("Gzmb+"), 
    msP_9=c("Prss34+"), 
    msP_10=c("Ms4a2+", "Gzmb+", "Prss34+", "Fcgr2b+", "Fcgr3+", "Cd34+"), 
    msP_11=c("Cd9+"), 
    msP_12=c("Ly6a+"), 
    msP_13=c("Gata1+", "Klf1+", "Gata2-")


)

In [ ]:
so <- UCell::AddModuleScore_UCell(so, features=features, assay="RNA", slot="counts", name="", maxRank=2500)

In [ ]:
pt_size <- 0.25
order <- TRUE

max_set <- 1
min_cutoff <- 0
max_cutoff <- 1

In [ ]:
p_l <- list(

    dplot(so, group_by="celltype_low", group_color=color$celltype_low, reduction="CR", pt_size=pt_size, legend_position="none", size_select=4), 
    fplot(so, features="msP_1", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_2", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_3", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_4", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_5", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_6", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_7", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_8", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_9", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_10", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_11", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4), 
    fplot(so, features="msP_12", assay="RNA", slot="data", reduction="CR", pt_size=pt_size, order=order, max_set=max_set, min_cutoff=min_cutoff, max_cutoff=max_cutoff, size_select=4)
    
)

In [ ]:
p_l <- lapply(p_l, function(p) egg::set_panel_size(p, width=unit(3, "cm"), height=unit(3, "cm")))
p_l <- do.call(gridExtra::arrangeGrob, c(p_l, ncol=10, nrow=ceiling(length(p_l)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/cr_p_marker_genes.pdf", width=10*2, height=2*ceiling(length(p_l)/10))

grid::grid.draw(p_l)

dev.off()

# Progenitor distribution 

In [ ]:
data <- so@meta.data %>% dplyr::filter(celltype_low!="Proerythroblast")

In [ ]:
features <- list(

    msP_1=c("Hlf+", "Esam+"), 
    msP_2=c("Kit+", "Flt3+"), 
    msP_3=c("Cd24a+", "Tfrc+"), 
    msP_4=c("Slamf1+"),
    msP_5=c("Fcgr2b+", "Fcgr3+"), 
    msP_6=c("Ms4a2+"),  
    msP_7=c("Elane+", "Prtn3+"), 
    msP_8=c("Gzmb+"), 
    msP_9=c("Prss34+"), 
    msP_10=c("Ms4a2+", "Gzmb+", "Prss34+", "Fcgr2b+", "Fcgr3+", "Cd34+"), 
    msP_11=c("Cd9+"), 
    msP_12=c("Ly6a+"), 
    msP_13=c("Gata1+", "Klf1+", "Gata2-")

)

In [ ]:
# GMP Hlf+ Esam+
ratio <- data %>% dplyr::filter(celltype_low=="GMP") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_1>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP Hlf+ Esam+ 
ratio <- data %>% dplyr::filter(celltype_low=="MEP") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_1>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# Kit+ Flt3+ 
ratio <- data %>% dplyr::filter(celltype_low!="Proerythroblast") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_2>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP Cd24+ Tfrc+ 
ratio <- data %>% dplyr::filter(celltype_low=="MEP") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_3>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# Proerythroblast Cd24+ Tfrc+ 
ratio <- data %>% dplyr::filter(celltype_low=="Proerythroblast") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_3>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# Granulocyte Cd24+ Tfrc+ 
ratio <- data %>% dplyr::filter(!(celltype_low %in% c("MEP", "Proerythroblast"))) %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_3>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP Cd24+ Tfrc+ 
ratio <- data %>% dplyr::filter(celltype_low=="MEP") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_3>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP Cd24+ Tfrc+ with Slamf1
ratio <- data %>% dplyr::filter(msP_3>0 & msP_4>0) %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(celltype_low %in% c("MegP", "Proerythroblast", "MEP"))/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP with BMP 
ratio <- data %>% dplyr::filter(celltype_low=="MEP") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_10>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)
ratio

In [ ]:
# Slamf1 with granulocyte potential 
ratio <- data %>% dplyr::filter(msP_4>0 & celltype_low=="MEP") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_10>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# Slamf1 without granulocyte potential 
ratio <- data %>% dplyr::filter(msP_10==0 & celltype_low=="MEP") %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_4>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP without granulocyte potential at Ctl and D6 with Ly6a expression 
ratio <- data %>% dplyr::filter(msP_10==0 & celltype_low=="MEP" & sample_group %in% c("Bl6_NaCl_D6", "Bl6_CpG_D6", "IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0")) %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_12>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP without granulocyte potential at D1 and D3 with Ly6a expression 
ratio <- data %>% dplyr::filter(msP_10==0 & celltype_low=="MEP" & !(sample_group %in% c("Bl6_NaCl_D6", "Bl6_CpG_D6", "IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0"))) %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_12>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP without granulocyte potential at D1 and D3 with Ly6a expression 
ratio <- data %>% dplyr::filter(msP_10==0 & celltype_low=="MEP" & sample_group %in% c("IFNAR_fl_CpG_D3")) %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_12>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
# MEP without granulocyte potential at D1 and D3 with Ly6a expression 
ratio <- data %>% dplyr::filter(msP_10==0 & celltype_low=="MEP" & sample_group %in% c("IFNAR_fl_LysM_cre_CpG_D3")) %>% dplyr::group_by(sample_group) %>% dplyr::summarise(ratio=sum(msP_12>0)/n()) %>% dplyr::pull(ratio)
round(mean(100*ratio), digits=1)
round(sd(100*ratio), digits=1)

In [ ]:
data <- so@meta.data %>% dplyr::filter(celltype_low %in% c("MEP")) %>% 
    dplyr::mutate(group=ifelse(msP_10==0 & msP_4>0, "MEP Slamf1+", "Negative")) %>% 
    dplyr::mutate(group=ifelse(msP_10==0 & msP_4==0, "MEP Slamf1-", group)) %>% 
    dplyr::mutate(group=ifelse(msP_10>0 & msP_4>0, "BMP Slamf1+", group)) %>% 
    dplyr::mutate(group=ifelse(msP_10>0 & msP_4==0, "BMP Slamf1-", group)) %>% 
    dplyr::mutate(group=factor(group, levels=c("MEP Slamf1+", "MEP Slamf1-", "BMP Slamf1+", "BMP Slamf1-"))) %>% 
    dplyr::mutate(dpi=ifelse(sample_group=="Bl6_NaCl_D6", "D0", as.character(dpi)))

bp_1 <- ggplot(data, aes(x=genotype, fill=group)) + 
    geom_bar(position="fill") + 
    facet_grid(~dpi, space="free", scales="free") + 
    scale_fill_manual(values=c("MEP Slamf1+"="#ffcdb2", "MEP Slamf1-"="#ffb4a2", "BMP Slamf1+"="#83c5be", "BMP Slamf1-"="#006d77")) + 
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
data <- so@meta.data %>% dplyr::filter(celltype_low %in% c("MEP")) %>% dplyr::filter(msP_10==0) %>%
    dplyr::mutate(group=ifelse(msP_10==0 & msP_11>0 & msP_12>0, "MEP Cd9+ Ly6a+", "Negative")) %>% 
    dplyr::mutate(group=ifelse(msP_10==0 & msP_11==0 & msP_12>0, "MEP Cd9- Ly6a+", group)) %>% 
    dplyr::mutate(group=ifelse(msP_10==0 & msP_11>0 & msP_12==0, "MEP Cd9+ Ly6a-", group)) %>% 
    dplyr::mutate(group=ifelse(msP_10==0 & msP_11==0 & msP_12==0, "MEP Cd9- Ly6a-", group)) %>% 
    dplyr::mutate(group=ifelse(msP_10>0, "BMP", group)) %>% 
    dplyr::mutate(group=factor(group, levels=c("MEP Cd9+ Ly6a+", "MEP Cd9- Ly6a+", "MEP Cd9+ Ly6a-", "MEP Cd9- Ly6a-", "BMP"))) %>% 
    dplyr::mutate(dpi=ifelse(sample_group=="Bl6_NaCl_D6", "D0", as.character(dpi)))

bp_2 <- ggplot(data, aes(x=genotype, fill=group)) + 
    geom_bar(position="fill") + 
    facet_grid(~dpi, space="free", scales="free") + 
    scale_fill_manual(values=c("MEP Cd9+ Ly6a+"="#ffcdb2", "MEP Cd9- Ly6a+"="#ffb4a2", "MEP Cd9+ Ly6a-"="#e5989b", "MEP Cd9- Ly6a-"="#b5838d")) + 
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
data <- so@meta.data %>% 
    dplyr::mutate(group=ifelse(msP_3>0, "Cd24a+ Tfrc+", "Negative")) %>%
    dplyr::mutate(group=factor(group, levels=c("Cd24a+ Tfrc+", "Negative")))

bp_3 <- ggplot(data, aes(x=celltype_low, fill=group)) + 
    geom_bar(position="fill") + 
    scale_fill_manual(values=c("Cd24a+ Tfrc+"="#ffcdb2", "Negative"="#ffb4a2")) + 
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/bp_p_ratio.pdf", width=3, height=6)

gridExtra::grid.arrange(

    bp_1 %>% egg::set_panel_size(., width=unit(c(3*0.3, 2*0.3, 2*0.3, 0.3), "cm"), height=unit(2.0, "cm")),
    bp_2 %>% egg::set_panel_size(., width=unit(c(3*0.3, 2*0.3, 2*0.3, 0.3), "cm"), height=unit(2.0, "cm")),
    bp_3 %>% egg::set_panel_size(., width=unit(c(7*0.3), "cm"), height=unit(2.0, "cm")), ncol=1
        
)

dev.off()

# Cell cycle score 

In [ ]:
so <- readRDS("data/scRNAseq/object/pp.rds")
so <- NormalizeData(so)

In [ ]:
# so <- so[, so$celltype_low %in% c("MEP", "Proerythroblast", "Erythroblast")]

In [ ]:
cc_genes <- read.csv("data/reference/cellcycle/seurat.csv", row.names=1)

In [ ]:
features <- list(

    msG2M=cc_genes[cc_genes$group=="G2M", ]$gene, 
    msS=cc_genes[cc_genes$group=="S", ]$gene, 
    msCC=c("Mki67", "Topa2")

)

In [ ]:
so <- UCell::AddModuleScore_UCell(so, features=features, assay="RNA", slot="data", name="")

In [ ]:
so$cc_phase <- ifelse((so$msS>0 & so$msCC>0 & so$msS>so$msG2M), "S", ifelse((so$msG2M>0 & so$msCC>0 & so$msG2M>so$msS), "G2M", "G1")) 

In [ ]:
options(repr.plot.width=35, repr.plot.height=5)

ggplot(so@meta.data, aes(x=msCC, y=msS, color=msCC)) + geom_point() + theme(aspect.ratio=1) + theme_global_set(1) + facet_grid(~celltype_low)
ggplot(so@meta.data, aes(x=msCC, y=msG2M, color=msCC)) + geom_point() + theme(aspect.ratio=1) + theme_global_set(1) + facet_grid(~celltype_low)
ggplot(so@meta.data, aes(x=msG2M, y=msS, color=msCC)) + geom_point() + theme(aspect.ratio=1) + theme_global_set(1) + facet_grid(~celltype_low)
ggplot(so@meta.data, aes(x=msG2M, y=msS, color=cc_phase)) + geom_point() + scale_color_manual(values=color$cc_phase) + theme(aspect.ratio=1) + theme_global_set(1) + facet_grid(~celltype_low) 

In [ ]:
data <- so@meta.data %>% dplyr::filter(celltype_low %in% c("MEP")) %>% 
    dplyr::mutate(dpi=ifelse(sample_group=="Bl6_NaCl_D6", "D0", as.character(dpi))) %>% 
    dplyr::mutate(cc_phase=factor(cc_phase, levels=rev(c("G1", "S", "G2M"))))

bp_1 <- ggplot(data, aes(x=genotype, fill=cc_phase)) + 
    geom_bar(position="fill") + 
    facet_grid(~dpi, space="free", scales="free") + 
    scale_fill_manual(values=color$cc_phase) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
data <- so@meta.data %>% dplyr::filter(celltype_low %in% c("Proerythroblast")) %>% 
    dplyr::mutate(dpi=ifelse(sample_group=="Bl6_NaCl_D6", "D0", as.character(dpi))) %>% 
    dplyr::mutate(cc_phase=factor(cc_phase, levels=rev(c("G1", "S", "G2M"))))

bp_2 <- ggplot(data, aes(x=genotype, fill=cc_phase)) + 
    geom_bar(position="fill") + 
    facet_grid(~dpi, space="free", scales="free") + 
    scale_fill_manual(values=color$cc_phase) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
data <- so@meta.data %>% dplyr::filter(celltype_low %in% c("Erythroblast")) %>% 
    dplyr::mutate(dpi=ifelse(sample_group=="Bl6_NaCl_D6", "D0", as.character(dpi))) %>% 
    dplyr::mutate(cc_phase=factor(cc_phase, levels=rev(c("G1", "S", "G2M"))))

bp_3 <- ggplot(data, aes(x=genotype, fill=cc_phase)) + 
    geom_bar(position="fill") + 
    facet_grid(~dpi, space="free", scales="free") + 
    scale_fill_manual(values=color$cc_phase) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/bp_cc_phase.pdf", width=3, height=6)

gridExtra::grid.arrange(

    bp_1 %>% egg::set_panel_size(., width=unit(c(3*0.3, 2*0.3, 2*0.3, 0.3), "cm"), height=unit(2.0, "cm")),
    bp_2 %>% egg::set_panel_size(., width=unit(c(3*0.3, 2*0.3, 2*0.3, 0.3), "cm"), height=unit(2.0, "cm")), 
    bp_3 %>% egg::set_panel_size(., width=unit(c(3*0.3, 2*0.3, 2*0.3, 0.3), "cm"), height=unit(2.0, "cm")), ncol=1
        
)

dev.off()

# Transcription factor heatmap of smoothed expression 

## Load pruned graph TF and target gens 

In [ ]:
reg <- read.csv("result/scenic/scRNAseq/res_1/reg.csv", row.names=1, skip=1)
colnames(reg) <- c("TF", "target_genes")

In [ ]:
tf <- c("Nfkb1", "Myb", "Fli1", "Gata2", "E2f3", "Bcl11a", "Myc", "Ikzf1", "Zfp148", "Irf8", "Irf1", "Irf7", "Stat1", "Stat2", "Stat3", "Bclaf1", "E2f4", "Gata1", "E2f7", "Tal1", "E2f1", "Mybl2", "Klf1", "E2f8", "Klf3", "E2f2")

## PTCG Ctl (+/+) and D1 (+/+) and D3 (+/+)

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes_diff_1 <- intersect(ptpg_res_1[ptpg_res_1$ptpg_class=="Canonical", ]$gene, ptpg_res_2[ptpg_res_2$ptpg_class=="Canonical", ]$gene) # Union of response genes D1 (+/+) and D3 (+/+)
genes_diff_1 <- genes_diff_1[!genes_diff_1 %in% ptpg_res_3[ptpg_res_3$ptpg_class != "Canonical", ]$gene] # Remove genes that are perturbed between D1 (+/+) and D3 (+/+)
length(genes_diff_1)

In [ ]:
pt_split_ptcg <- pt_split(fitgam, genes_diff_1, contrast_vec)

In [ ]:
tf_tg_mat_ptcg <- reg[reg$target_genes %in% names(pt_split_ptcg), ]
tf_tg_mat_ptcg <- tf_tg_pp(tf_tg_mat_ptcg)
tf_tg_mat_ptcg <- tf_tg_prune(tf_tg_mat_ptcg, pt_split_ptcg, tf_ratio=0, tf_sum=0, tf_fill=TRUE)
tf_tg_mat_ptcg <- tf_tg_mat_ptcg[, tf]

In [ ]:
pt_tf_hm_1 <- pt_tf_hm(tf_tg_mat_ptcg, row_split=pt_split_ptcg, height=180, fontsize_select=2, cluster_rows=FALSE, cluster_columns=FALSE, use_raster=TRUE)
pt_hm_1 <- pt_hm(fitgam, names(pt_split_ptcg), "IFNAR_fl_Baseline_D0", contrast_vec, 50, color_mat[1], row_split=pt_split_ptcg, width=0.75, height=180, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_ptcg), "IFNAR_fl_CpG_D1", contrast_vec, 50, color_mat[2], row_split=pt_split_ptcg, width=0.75, height=180, fontsize_select=2, use_raster=TRUE)
pt_hm_3 <- pt_hm(fitgam, names(pt_split_ptcg), "IFNAR_fl_CpG_D3", contrast_vec, 50, color_mat[3], row_split=pt_split_ptcg, width=0.75, height=180, fontsize_select=2, use_raster=TRUE)

In [ ]:
row_genes <- c(

    "Meis1", "Klf1", "Tal1", "Gata1", "Gata2", # Transcription factors 
    "Mki67", "Top2a", # Proliferation 
    "Pkm", # Glycolysis
    "Pdk1", # Glycolysis/aerobic metabolism
    "Hif1a", "Ldha", # Glycolysis/anaerboic metabolism
    "Atp5a1", "Atp5b", # Oxidative phosphorylation 
    "Sod2", # ROS detoxifiyer
    "G6pd", "Pgd", # PP
    "Cpt1a", "Acadl", # b-oxidatse
    "Mtor", "Rptor", "Rheb",  # mTOR pathwy
    "Tfam", "Ssbp1", "Polg", # Mitochondrial maintainance
    "Bnip3l", # Mitochondrial removal 
    "Dnmt1", "Dnmt3b", "Tet2", "Tet3", # Epigenetics
    "Alas2", "Ncoa4", # Heme metabolism 
    "Icam4", "Epor", "Tfrc" # Receptor

)

row_genes <- row_genes[row_genes %in% rownames(pt_tf_hm_1@matrix)]

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        
        at=which(rownames(pt_tf_hm_1@matrix) %in% row_genes),
        labels=rownames(pt_tf_hm_1@matrix)[which(rownames(pt_tf_hm_1@matrix) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/hm_tf_ptcg_ctl_d1_d3.pdf", width=5, height=10)

draw(row_anno + pt_tf_hm_1 + pt_hm_1 + pt_hm_2 + pt_hm_3)

dev.off()

## PTPG D1 (+/+) or D3 (+/+) UP vs Ctl (+/+)

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes_diff_1 <- intersect(ptpg_res_1[ptpg_res_1$ptpg_class=="IFNAR_fl_CpG_D1", ]$gene, ptpg_res_2[ptpg_res_2$ptpg_class=="IFNAR_fl_CpG_D3", ]$gene) # Union of response genes UP D1 (+/+) and D3 (+/+) vs Ctl (+/+)
length(genes_diff_1)

In [ ]:
pt_split_ptpg <- pt_split(fitgam, genes_diff_1, contrast_vec)

In [ ]:
tf_tg_mat_ptcg <- reg[reg$target_genes %in% names(pt_split_ptpg), ]
tf_tg_mat_ptcg <- tf_tg_pp(tf_tg_mat_ptcg)
tf_tg_mat_ptcg <- tf_tg_prune(tf_tg_mat_ptcg, pt_split_ptpg, tf_ratio=0, tf_sum=0, tf_fill=TRUE)
tf_tg_mat_ptcg[tf[!tf %in% colnames(tf_tg_mat_ptcg)]] <- 0
tf_tg_mat_ptcg <- tf_tg_mat_ptcg[, tf]

In [ ]:
pt_tf_hm_1 <- pt_tf_hm(tf_tg_mat_ptcg, row_split=pt_split_ptpg, height=50, fontsize_select=2, cluster_rows=FALSE, cluster_columns=FALSE, use_raster=FALSE)
pt_hm_1 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_Baseline_D0", contrast_vec, 50, color_mat[1], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_CpG_D1", contrast_vec, 50, color_mat[2], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)
pt_hm_3 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_CpG_D3", contrast_vec, 50, color_mat[3], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)

In [ ]:
row_genes <- c(
    
    "Meis1", "Klf1", "Tal1", "Gata1", "Gata2", # Transcription factors 
    "Mki67", "Top2a", # Proliferation 
    "Pkm", # Glycolysis
    "Pdk1", # Glycolysis/aerobic metabolism
    "Hif1a", "Ldha", # Glycolysis/anaerboic metabolism
    "Atp5a1", "Atp5b", # Oxidative phosphorylation 
    "Sod2", # ROS detoxifiyer
    "G6pd", "Pgd", # PP
    "Cpt1a", "Acadl", # b-oxidatse
    "Mtor", "Rptor", "Rheb",  # mTOR pathwy
    "Tfam", "Ssbp1", "Polg", # Mitochondrial maintainance
    "Bnip3l", # Mitochondrial removal 
    "Dnmt1", "Dnmt3b", "Tet2", "Tet3", # Epigenetics
    "Alas2", "Ncoa4", # Heme metabolism 
    "Icam4", "Epor", "Tfrc", # Receptor
    "Kit", "Cd24a", "Tfrc", "Slamf1", "Cd9", "Ly6a", 
    grep("Isg|Ifi|Irf|Socs|Smad|Nfkb|Stat|Icam|Smad|Bmp", genes_diff_1, value=TRUE), 
    grep("Tlr[0-9]+|Nlrp|Nlrc|Nod|Ddx58|Ifih1|Clec[0-9]+|Fc[grme]|Ifnar|Ifngr|Ifnlr|Il[0-9]+r[a-z]?|Ccr[0-9]+|Cxcr[0-9]+|Cx3cr1|Xcr1|Cd3|Cd79|Klra|Klrb|Klrc|Klrd|Klre|Klrf|Klrg|Kit|Cd[0-9]+[a-zA-Z]?|Tgfb[123]|Tgfbr[123]|Smad[1-9]|Bmp[0-9]+|Bmpr1a|Bmpr1b|Bmpr2|Acvr1|Acvr2a|Acvr2b|Gdf[0-9]+|Kitl|Kit|Nr3c1|Ptch[12]|Smo|Cdon|Boc|Gas1", genes_diff_1, value=TRUE)

)

row_genes <- row_genes[row_genes %in% rownames(pt_tf_hm_1@matrix)]

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        at=which(rownames(pt_tf_hm_1@matrix) %in% row_genes),
        labels=rownames(pt_tf_hm_1@matrix)[which(rownames(pt_tf_hm_1@matrix) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/hm_tf_ptpg_d1_or_d3_up_vs_ctl.pdf", width=6, height=10)

draw(row_anno + pt_tf_hm_1 + pt_hm_1 + pt_hm_2 + pt_hm_3)

dev.off()

## PTPG D1 (+/+) or D3 (+/+) DOWN vs Ctl (+/+)

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes_diff_1 <- union(ptpg_res_1[ptpg_res_1$ptpg_class=="IFNAR_fl_Baseline_D0", ]$gene, ptpg_res_2[ptpg_res_2$ptpg_class=="IFNAR_fl_Baseline_D0", ]$gene) # Union of response genes DOWN D1 (+/+) and D3 (+/+) vs Ctl (+/+)
length(genes_diff_1)

In [ ]:
pt_split_ptpg <- pt_split(fitgam, genes_diff_1, contrast_vec)

In [ ]:
tf_tg_mat_ptcg <- reg[reg$target_genes %in% names(pt_split_ptpg), ]
tf_tg_mat_ptcg <- tf_tg_pp(tf_tg_mat_ptcg)
tf_tg_mat_ptcg <- tf_tg_prune(tf_tg_mat_ptcg, pt_split_ptpg, tf_ratio=0, tf_sum=0, tf_fill=TRUE)
tf_tg_mat_ptcg[tf[!tf %in% colnames(tf_tg_mat_ptcg)]] <- 0
tf_tg_mat_ptcg <- tf_tg_mat_ptcg[, tf]

In [ ]:
pt_tf_hm_1 <- pt_tf_hm(tf_tg_mat_ptcg, row_split=pt_split_ptpg, height=50, fontsize_select=2, cluster_rows=FALSE, cluster_columns=FALSE, use_raster=FALSE)
pt_hm_1 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_Baseline_D0", contrast_vec, 50, color_mat[1], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_CpG_D1", contrast_vec, 50, color_mat[2], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)
pt_hm_3 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_CpG_D3", contrast_vec, 50, color_mat[3], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)

In [ ]:
row_genes <- rownames(pt_tf_hm_1@matrix)

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        at=which(rownames(pt_tf_hm_1@matrix) %in% row_genes),
        labels=rownames(pt_tf_hm_1@matrix)[which(rownames(pt_tf_hm_1@matrix) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/hm_tf_ptpg_d1_or_d3_down_vs_ctl.pdf", width=6, height=10)

draw(row_anno + pt_tf_hm_1 + pt_hm_1 + pt_hm_2 + pt_hm_3)

dev.off()

## PTPG D1 (+/+) UP vs D3 (+/+) 

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes_diff_1 <- ptpg_res_3[ptpg_res_3$ptpg_class=="IFNAR_fl_CpG_D1", ]$gene # Genes up on D1 (+/+) vs D3 (+/+)
length(genes_diff_1)

In [ ]:
pt_split_ptpg <- pt_split(fitgam, genes_diff_1, contrast_vec)

In [ ]:
tf_tg_mat_ptcg <- reg[reg$target_genes %in% names(pt_split_ptpg), ]
tf_tg_mat_ptcg <- tf_tg_pp(tf_tg_mat_ptcg)
tf_tg_mat_ptcg <- tf_tg_prune(tf_tg_mat_ptcg, pt_split_ptpg, tf_ratio=0, tf_sum=0, tf_fill=TRUE)
tf_tg_mat_ptcg <- tf_tg_mat_ptcg[, colnames(tf_tg_mat_ptcg) %in% tf]

In [ ]:
tf_fill <- setdiff(tf, colnames(tf_tg_mat_ptcg))
tf_tg_mat_ptcg[, tf_fill] <- 0
tf_tg_mat_ptcg <- tf_tg_mat_ptcg[, tf, drop=FALSE]

In [ ]:
pt_tf_hm_1 <- pt_tf_hm(tf_tg_mat_ptcg, row_split=pt_split_ptpg, height=50, fontsize_select=2, cluster_rows=FALSE, cluster_columns=FALSE, use_raster=FALSE)
pt_hm_1 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_Baseline_D0", contrast_vec, 50, color_mat[1], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_CpG_D1", contrast_vec, 50, color_mat[2], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)
pt_hm_3 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_CpG_D3", contrast_vec, 50, color_mat[3], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)

In [ ]:
row_genes <- rownames(pt_tf_hm_1@matrix)

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        at=which(rownames(pt_tf_hm_1@matrix) %in% row_genes),
        labels=rownames(pt_tf_hm_1@matrix)[which(rownames(pt_tf_hm_1@matrix) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/hm_tf_ptpg_d1_up_vs_d3.pdf", width=6, height=10)

draw(row_anno + pt_tf_hm_1 + pt_hm_1 + pt_hm_2 + pt_hm_3)

dev.off()

## PTPG D1 (+/+) DOWN vs D3 (+/+) 

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes_diff_1 <- ptpg_res_3[ptpg_res_3$ptpg_class=="IFNAR_fl_CpG_D3", ]$gene # Genes up on D1 (+/+) vs D3 (+/+)
length(genes_diff_1)

In [ ]:
pt_split_ptpg <- pt_split(fitgam, genes_diff_1, contrast_vec)

In [ ]:
tf_tg_mat_ptcg <- reg[reg$target_genes %in% names(pt_split_ptpg), ]
tf_tg_mat_ptcg <- tf_tg_pp(tf_tg_mat_ptcg)
tf_tg_mat_ptcg <- tf_tg_prune(tf_tg_mat_ptcg, pt_split_ptpg, tf_ratio=0, tf_sum=0, tf_fill=TRUE)
tf_tg_mat_ptcg <- tf_tg_mat_ptcg[, colnames(tf_tg_mat_ptcg) %in% tf]

In [ ]:
tf_fill <- setdiff(tf, colnames(tf_tg_mat_ptcg))
tf_tg_mat_ptcg[, tf_fill] <- 0
tf_tg_mat_ptcg <- tf_tg_mat_ptcg[, tf, drop=FALSE]

In [ ]:
pt_tf_hm_1 <- pt_tf_hm(tf_tg_mat_ptcg, row_split=pt_split_ptpg, height=50, fontsize_select=2, cluster_rows=FALSE, cluster_columns=FALSE, use_raster=FALSE)
pt_hm_1 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_Baseline_D0", contrast_vec, 50, color_mat[1], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_CpG_D1", contrast_vec, 50, color_mat[2], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)
pt_hm_3 <- pt_hm(fitgam, names(pt_split_ptpg), "IFNAR_fl_CpG_D3", contrast_vec, 50, color_mat[3], row_split=pt_split_ptpg, width=0.75, height=50, fontsize_select=2, use_raster=TRUE)

In [ ]:
row_genes <- rownames(pt_tf_hm_1@matrix)

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        at=which(rownames(pt_tf_hm_1@matrix) %in% row_genes),
        labels=rownames(pt_tf_hm_1@matrix)[which(rownames(pt_tf_hm_1@matrix) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/hm_tf_ptpg_d1_down_vs_d3.pdf", width=6, height=10)

draw(row_anno + pt_tf_hm_1 + pt_hm_1 + pt_hm_2 + pt_hm_3)

dev.off()

# Line plots smoothed expression 

# Stress erythroid marker 

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes <- c("Kit", "Cd24a", "Tfrc", "Slamf1", "Cd9", "Ly6a")

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt"), condition_line_type=TRUE) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/lp_stress_erythroid_marker.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

## PTCG Ctl (+/+) and D1 (+/+) and D3 (+/+) 

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes <- c("Meis1", "Klf1", "Tal1", "Gata1", "Epor", "Tfrc", "Alas2")
genes %in% intersect(ptpg_res_1[ptpg_res_1$ptpg_class=="Canonical", ]$gene, ptpg_res_2[ptpg_res_2$ptpg_class=="Canonical", ]$gene)

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt"), condition_line_type=FALSE) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/lp_ptcg_ctl_d1_d3.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

## PTPG D1 (+/+) or D3 (+/+) UP vs Ctl (+/+)

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes <- union(ptpg_res_1[ptpg_res_1$ptpg_class=="IFNAR_fl_CpG_D1" , ]$gene, ptpg_res_2[ptpg_res_2$ptpg_class=="IFNAR_fl_CpG_D3" , ]$gene)
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt"), condition_line_type=TRUE) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/lp_ptpg_d1_or_d3_up_vs_ctl.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

## PTPG D1 (+/+) or D3 (+/+) DOWN vs Ctl (+/+)

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes <- union(ptpg_res_1[ptpg_res_1$ptpg_class=="IFNAR_fl_Baseline_D0" , ]$gene, ptpg_res_2[ptpg_res_2$ptpg_class=="IFNAR_fl_Baseline_D0" , ]$gene)
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt"), condition_line_type=TRUE) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/lp_ptpg_d1_or_d3_down_vs_ctl.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

## PTPG D1 (+/+) or D3 (+/+) vs Ctl (+/+)

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes <- union(ptpg_res_1[ptpg_res_1$ptpg_class=="Perturbed" , ]$gene, ptpg_res_2[ptpg_res_2$ptpg_class=="Perturbed" , ]$gene)
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [ ]:
# lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt"), condition_line_type=TRUE) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
# lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
# lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
# pdf("result/figures/1_scRNAseq/panel_4/lp_ptpg_d1_or_d3_vs_ctl.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

# grid::grid.draw(lp_1)

# dev.off()

## PTPG D1 (+/+) UP vs D3 (+/+) 

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes <- ptpg_res_3[ptpg_res_3$ptpg_class=="IFNAR_fl_CpG_D1" , ]$gene
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt"), condition_line_type=TRUE) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/lp_ptpg_d1_up_vs_d3.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

## PTPG D1 (+/+) DOWN vs D3 (+/+) 

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes <- ptpg_res_3[ptpg_res_3$ptpg_class=="IFNAR_fl_CpG_D3" , ]$gene
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt"), condition_line_type=TRUE) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/lp_ptpg_d1_down_vs_d3.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

## PTPG D1 (+/+) vs D3 (+/+)

In [ ]:
contrast_vec <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")
color_mat <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]

In [ ]:
genes <- ptpg_res_3[ptpg_res_3$ptpg_class=="Perturbed" , ]$gene
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [ ]:
# lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt"), condition_line_type=TRUE) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
# lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
# lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
# pdf("result/figures/1_scRNAseq/panel_4/lp_ptpg_d1_vs_d3.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

# grid::grid.draw(lp_1)

# dev.off()

# Pie chart PT gene counts

In [ ]:
ptpg_class_color <- c("Perturbed"="#7f7f7f", "Canonical"="#AEAEAE")
data <- table(ptpg_res_1 %>% dplyr::mutate(ptpg_class=ifelse(ptpg_class=="Canonical", "Canonical", "Perturbed")) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_1 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctl (+/+) vs D1 (+/+)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
ptpg_class_color <- c("Perturbed"="#7f7f7f", "Canonical"="#AEAEAE")
data <- table(ptpg_res_2 %>% dplyr::mutate(ptpg_class=ifelse(ptpg_class=="Canonical", "Canonical", "Perturbed")) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_2 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctl (+/+) vs D1 (+/+)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
ptpg_class_color <- c("Perturbed"="#7f7f7f", "Canonical"="#AEAEAE")
data <- table(ptpg_res_3 %>% dplyr::mutate(ptpg_class=ifelse(ptpg_class=="Canonical", "Canonical", "Perturbed")) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_3 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctl (+/+) vs D1 (+/+)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
ptpg_class_color <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1")]
data <- table(ptpg_res_1 %>% dplyr::filter(ptpg_class %in% names(ptpg_class_color)) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_4 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctl (+/+) vs D1 (+/+)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
ptpg_class_color <- color$sample_group[c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D3")]
ptpg_class_color[2] <- scales::alpha(ptpg_class_color[2], 0.5)
data <- table(ptpg_res_2 %>% dplyr::filter(ptpg_class %in% names(ptpg_class_color)) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_5 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctl (+/+) vs D1 (+/+)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
ptpg_class_color <- color$sample_group[c("IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3")]
ptpg_class_color[2] <- scales::alpha(ptpg_class_color[2], 0.5)
data <- table(ptpg_res_3 %>% dplyr::filter(ptpg_class %in% names(ptpg_class_color)) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_6 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctl (+/+) vs D1 (+/+)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_4/p_pl_ptg_counts.pdf", width=5, height=7.5)

gridExtra::grid.arrange(
    
    p_pl_1 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    p_pl_2 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    p_pl_3 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    p_pl_4 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    p_pl_5 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    p_pl_6 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    ncol=2
    
)

dev.off()